In [1]:
from ultralytics import YOLO
import cv2

import time
import serial
import serial.tools.list_ports
import sys

In [2]:
model = YOLO("gc10-det_openvino_model", task = "detect")

In [3]:
def startSerial():
    ser = None
    ports = serial.tools.list_ports.comports()
    for port, desc, hwid in sorted(ports):
        if 'USB' in desc:
            print('Microbit USB detected: ', port)
            ser = serial.Serial(port, 115200, timeout = 0, parity=serial.PARITY_NONE, rtscts=0)
    return ser

def SerialSendCommand(cmd, ser):
    if ser is not None:
        cmd = str(cmd)
        cmd = cmd + '\n'
        cmd = str.encode(cmd)
        ser.write(cmd)

In [4]:
def Main():
    camera = cv2.VideoCapture(source)
    bg = cv2.imread('data/background.jpg')
    color = (0, 255, 0)
    ser = startSerial()
    tic = time.time()

    while(True):
        ret, frame = camera.read()
        if not ret:
            break

        results = model(frame, verbose = False, conf = .45)
        output_image = results[0].plot()
        output_image = cv2.resize(output_image, (1024, 500))

        if len(results[0].boxes.cls) > 0:
            color = (0, 0, 255)
            cmd = 1
        else:
            color = (0, 255, 0)
            cmd = 0

        if time.time()-tic>1:
            tic=time.time()
            SerialSendCommand(cmd,ser)

        cv2.namedWindow("Quality Control", cv2.WINDOW_NORMAL)
        cv2.setWindowProperty("Quality Control", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        bg[20:520, 130:1154] = output_image
        cv2.rectangle(bg, (130, 20), (1154, 520), color, 2)
        cv2.imshow("Quality Control", bg)

        if cv2.waitKey(1) & 0xff == ord(' '):
            break

    if ser is not None:
        ser.close()
 
    camera.release()
    cv2.destroyAllWindows()

In [6]:
source = "data/test.mp4"
if __name__ == '__main__':
    Main()

Microbit USB detected:  COM3
